In [ ]:
import sys
sys.path.insert(1, '../scr')
from PhraseDictionary import PhraseDictionary
from toolbox import Configurator, AnkiCommunicator, AnkiCardWriter
from DictionaryReader import CollinsReader, PhraseReader
from ExerciseWriter import ExerciseWriter
from Exercise import ExerciseFactory, ExerciseGatherer
import pyperclip
from pathlib import Path
import shutil
import json


def copy_folder(source_folder, destination_folder) -> None:
    destination_path = Path(destination_folder)
    source_path = Path(source_folder)
    if not destination_path.exists():
        shutil.copytree(source_path, destination_path)

### Initial set-up

In [ ]:
# User inputs
date = 20240825


# Make a new folder
template_folder = '../scr/resources/Templates'
export_folder = f'./Exports/{date}'
copy_folder(template_folder, export_folder)

# Paths of the templates
def_template_path = '/Definition/template.tex'
equivalence_template_path = '/Equivalence/template.tex'
reading_template_path = '/Reading/template.tex'
cloze_template_path = '/Cloze/template.tex'
gap_filling_template_path = '/Fill-in-the-gap/template.tex'
paraphrase_template_path = '/Paraphrase/template.tex'
dialogue_template_path = '/Dialogue/template.tex'

# Paths of the output folder
def_output_folder = export_folder + '/Definition'
equivalence_output_folder = export_folder + '/Equivalence'
reading_output_folder = export_folder + '/Reading'
cloze_output_folder = export_folder + '/Cloze'
gap_filling_output_folder = export_folder + '/Fill-in-the-gap'
paraphrase_output_folder = export_folder + '/Paraphrase'
dialogue_output_folder = export_folder + '/Dialogue'

# Creation of writers
factory = ExerciseFactory()
def_gatherer = ExerciseGatherer()
def_writer = ExerciseWriter(gatherer=def_gatherer, template_path=def_template_path, output_folder=def_output_folder)
equivalence_gatherer = ExerciseGatherer()
equivalence_writer = ExerciseWriter(gatherer=equivalence_gatherer, template_path=equivalence_template_path, output_folder=equivalence_output_folder)
cloze_gatherer = ExerciseGatherer()
cloze_exercise_writer = ExerciseWriter(gatherer=cloze_gatherer, template_path=cloze_template_path, output_folder=cloze_output_folder)
reading_gatherer = ExerciseGatherer()
reading_exercise_writer = ExerciseWriter(gatherer=reading_gatherer, template_path=reading_template_path, output_folder=reading_output_folder)
gap_filling_gatherer = ExerciseGatherer()
gap_filling_exercise_writer = ExerciseWriter(gatherer=gap_filling_gatherer, template_path=gap_filling_template_path, output_folder=gap_filling_output_folder)
paraphrase_gatherer = ExerciseGatherer()
paraphrase_exercise_writer = ExerciseWriter(gatherer=paraphrase_gatherer, template_path=paraphrase_template_path, output_folder=paraphrase_output_folder)
dialogue_gatherer = ExerciseGatherer()
dialogue_exercise_writer = ExerciseWriter(gatherer=dialogue_gatherer, template_path=dialogue_template_path, output_folder=dialogue_output_folder)

### Configuration for `Vocab Explorer`

In [ ]:
category = 'Vocab Explorer'
# category = 'Vocab Builder'
num_of_words_to_learn = 15

configurator = Configurator(json_path=f'../scr/config/{category}/config.json')
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
reader = CollinsReader(word_list=tomorrow_new)
entry_list = reader.get_word_entry_list()
writer = AnkiCardWriter(entry_list)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')
print('Go to Anki and import the cards from the file Anki Cards.csv')
configurator.study_n_words(num_of_words_to_learn)

In [ ]:
anki = AnkiCommunicator()
tomorrow_new_with_def = reader.get_concise_dictionary()
tomorrow_review_with_def = anki.get_words_for_tomorrow(deck_name=category)
tomorrow_review = list(tomorrow_review_with_def.keys())
print(f'''
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')

### Configuration for `Everyday English`

In [ ]:
category = 'Everyday English'
num_of_words_to_learn = 10

configurator = Configurator(json_path=f'../scr/config/{category}/config.json')
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
reader = PhraseReader(word_list=tomorrow_new)
entry_list = reader.get_word_entry_list()
writer = AnkiCardWriter(entry_list)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')
print('Go to Anki and import the cards from the file Anki Cards.csv')
configurator.study_n_words(num_of_words_to_learn)

In [ ]:
anki = AnkiCommunicator()
tomorrow_new_with_def = reader.get_concise_dictionary()
tomorrow_review_with_def = anki.get_words_for_tomorrow(deck_name=category)
tomorrow_review = list(tomorrow_review_with_def.keys())
print(f'''
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')

### 1. Definition

#### Set the word list and the set index here and get the prompt to create definitions

In [ ]:
def_start_idx = 9
def_end_idx = 15
def_set_index = 2

def_word_list = tomorrow_new[def_start_idx-1:def_end_idx]
definitions_set = factory.create_exercise(exercise_type='Definition', word_entries={word: [""] for word in def_word_list})
definitions_set.import_definition_from_dictionary()
definitions_set.finish_import()
def_gatherer.import_exercise(definitions_set)
def_writer.render_template(set_index=def_set_index)

### 2. Equivalence

#### Enter the index

In [ ]:
equivalence_start_idx = 21
equivalence_end_idx = 30
equivalence_set_index = 3

equivalence_word_list = tomorrow_review[equivalence_start_idx-1:equivalence_end_idx]
equivalence_exercise = factory.create_exercise(exercise_type='Equivalence', 
                                                         word_entries={term: tomorrow_review_with_def[term] for term in equivalence_word_list})

equivalence_exercise.get_prompt()

In [ ]:
equivalence_exercise_text = pyperclip.paste()
equivalence_exercise.import_exercise(text=equivalence_exercise_text)
equivalence_exercise.finish_import()
equivalence_gatherer.import_exercise(equivalence_exercise)
equivalence_writer.render_template(equivalence_set_index)

### 3. Reading

#### Enter the set index and the word list here

In [ ]:
reading_start_idx = 1
reading_end_idx = 10
reading_set_index = 1
reading_word_list = tomorrow_review[reading_start_idx-1:reading_end_idx]
reading_exercise = factory.create_exercise(exercise_type='Reading', word_entries={term: tomorrow_review_with_def[term] for term in reading_word_list})
reading_exercise.get_prompt()

#### Paste the passages here

In [ ]:
passages = pyperclip.paste()
reading_exercise.import_passage(text=passages)
reading_exercise.get_second_prompt()

#### Paste the cloze exercise here

In [ ]:
cloze_exercise_text = pyperclip.paste()

reading_exercise.import_exercise(text=cloze_exercise_text)
reading_exercise.finish_import()
reading_gatherer.import_exercise(exercise=reading_exercise)
reading_exercise_writer.render_template(set_index=reading_set_index)

### 4. Fill in the gaps

#### Set the word list here and get the prompt to create example sentences

In [ ]:
gap_filling_start_idx = 9
gap_filling_end_idx = 15
gap_filling_set_index = 7

vocab_word_list = tomorrow_new[gap_filling_start_idx-1:gap_filling_end_idx]
vocab_word_entries = {term: tomorrow_new_with_def[term] for term in vocab_word_list}

# vocab_word_list = tomorrow_review[gap_filling_start_idx-1:gap_filling_end_idx]
# vocab_word_entries = {term: tomorrow_review_with_def[term] for term in vocab_word_list}


gap_filling_exercise = factory.create_exercise(exercise_type='Fill in the gap', word_entries=vocab_word_entries)
gap_filling_exercise.get_prompt()

#### Paste the example sentences here

In [ ]:
sentences = pyperclip.paste()
gap_filling_exercise.import_sentences(text=sentences)
gap_filling_exercise.finish_import()
gap_filling_gatherer.import_exercise(exercise=gap_filling_exercise)
gap_filling_exercise_writer.render_template(set_index=gap_filling_set_index)

### 5. Dialogue

#### Enter the set index here

In [ ]:
dialogue_start_idx = 7
dialogue_end_idx = 13
dialogue_set_index = 2

# dialogue_word_list = tomorrow_new[dialogue_start_idx-1:dialogue_end_idx]
dialogue_word_list = tomorrow_review[dialogue_start_idx-1:dialogue_end_idx]
dialogue_exercise = factory.create_exercise(exercise_type='Dialogue', word_entries={term: tomorrow_review_with_def[term] for term in dialogue_word_list})
dialogue_exercise.get_prompt()

In [ ]:
dialogue_dict_text = pyperclip.paste()
dialogue_dict = json.loads(dialogue_dict_text)
dialogue_exercise.generate_exercise(dialogue_dict=dialogue_dict)
dialogue_exercise.finish_import()
dialogue_gatherer.import_exercise(exercise=dialogue_exercise)
dialogue_exercise_writer.render_template(set_index=dialogue_set_index)

### 6. Cloze

#### Enter the set index and the word list here

In [ ]:
cloze_start_idx = 21
cloze_end_idx = 23
cloze_set_index = 3
cloze_word_list = tomorrow_review[cloze_start_idx-1:cloze_end_idx]
cloze_exercise = factory.create_exercise(exercise_type='Cloze', word_entries={term: tomorrow_review_with_def[term] for term in cloze_word_list})
cloze_exercise.get_prompt()

#### Paste the passages here

In [ ]:
passages = pyperclip.paste()
cloze_exercise.import_passage(text=passages)
cloze_exercise.get_second_prompt()

#### Paste the cloze exercise here

In [ ]:
cloze_exercise_text = pyperclip.paste()

cloze_exercise.import_exercise(text=cloze_exercise_text)
cloze_exercise.finish_import()
cloze_gatherer.import_exercise(exercise=cloze_exercise)
cloze_exercise_writer.render_template(set_index=cloze_set_index)